In [26]:
import pandas as pd

df_01 = pd.read_excel('텍사스주_우물정보_25_02.xls', sheet_name= '01')
df_02 = pd.read_excel('텍사스주_우물정보_25_02.xls', sheet_name= '02')
df_03 = pd.read_excel('텍사스주_우물정보_25_02.xls', sheet_name= '03')
df_04 = pd.read_excel('텍사스주_우물정보_25_02.xls', sheet_name= '04')
df_05 = pd.read_excel('텍사스주_우물정보_25_02.xls', sheet_name= '05')
df_06 = pd.read_excel('텍사스주_우물정보_25_02.xls', sheet_name= '06')
df_08 = pd.read_excel('텍사스주_우물정보_25_02.xls', sheet_name= '08')
df_09 = pd.read_excel('텍사스주_우물정보_25_02.xls', sheet_name= '09')
df_10 = pd.read_excel('텍사스주_우물정보_25_02.xls', sheet_name= '10')
df_11 = pd.read_excel('텍사스주_우물정보_25_02.xls', sheet_name= '7B')
df_12 = pd.read_excel('텍사스주_우물정보_25_02.xls', sheet_name= '7C')
df_13 = pd.read_excel('텍사스주_우물정보_25_02.xls', sheet_name= '8A')

# Combine all the dataframes into one
df = pd.concat([df_01, df_02, df_03, df_04, df_05, df_06, df_08, df_09, df_10, df_11, df_12, df_13], ignore_index=True)

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1239 entries, 0 to 1238
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   District               1239 non-null   object 
 1   API Number             1239 non-null   int64  
 2   Field Name             1198 non-null   object 
 3   Field Number           1198 non-null   float64
 4   Lease/Gas ID           1235 non-null   float64
 5   Lease Name             1239 non-null   object 
 6   Well Number            1200 non-null   object 
 7   Operator Name          1239 non-null   object 
 8   County Name            1239 non-null   object 
 9   County Number          1239 non-null   int64  
 10  Survey Name            1239 non-null   object 
 11  Survey Section Number  956 non-null    float64
 12  Alternate Survey Name  0 non-null      float64
 13  Abstract Number        1190 non-null   object 
 14  Block                  782 non-null    object 
 15  O/G/

In [ ]:
# 데이터 열 고르기
selected_columns = df[['API Number', 'County Name', 'County Number', 'Bottom Total Depth']]
selected_columns.head()

,API Number,County Name,County Number,Bottom Total Depth
0,1335890,ATASCOSA,13,17409
1,47944500,WEBB,479,20012
2,47944499,WEBB,479,19340
3,12738440,DIMMIT,127,17485
4,12738426,DIMMIT,127,15622


In [27]:
# county 값 갯수 카운트
selected_columns['County Name'].value_counts()

County Name
MIDLAND       88
REEVES        82
MARTIN        79
HOWARD        78
COCHRAN       65
              ..
PALO PINTO     1
DENTON         1
COTTLE         1
FOARD          1
CROSBY         1
Name: count, Length: 95, dtype: int64

In [29]:
# county 기준으로 내림차순 정렬
sorted_selected_columns = selected_columns.sort_values(by=['County Name', 'Bottom Total Depth'], ascending=[True, False])
sorted_selected_columns.head()


,API Number,County Name,County Number,Bottom Total Depth
193,132842,ANDERSON,1,12064
194,132842,ANDERSON,1,12064
191,132854,ANDERSON,1,9342
192,132854,ANDERSON,1,9342
734,348418,ANDREWS,3,20419


In [ ]:
# county 마다 평균 깊이
county_avg_depth = sorted_selected_columns.groupby('County Name')['Bottom Total Depth'].mean()


County Name
ANDERSON    10703.000000
ANDREWS     11747.500000
ARCHER       4506.333333
ATASCOSA    17589.714286
BEE         23334.000000
                ...     
WINKLER      9490.846154
WISE        14551.125000
YOAKUM       8151.222222
YOUNG        2731.538462
ZAVALA       1855.000000
Name: Bottom Total Depth, Length: 95, dtype: float64

In [ ]:
# county 별 갯수와 평균 깊이를 계산(최종 데이터 프레임)
county_stats = sorted_selected_columns.groupby(['County Name', 'County Number']).agg(
    County_Count=('County Name', 'size'),
    County_Avg_Depth=('Bottom Total Depth', 'mean')
).reset_index()

county_c_a = county_stats[['County Name', 'County_Count', 'County_Avg_Depth']]
 
county_c_a

,County Name,County Number,County_Count,County_Avg_Depth
0,ANDERSON,1,4,10703.000000
1,ANDREWS,3,16,11747.500000
2,ARCHER,9,18,4506.333333
3,ATASCOSA,13,14,17589.714286
4,BEE,25,2,23334.000000
...,...,...,...,...
90,WINKLER,495,13,9490.846154
91,WISE,497,8,14551.125000
92,YOAKUM,501,9,8151.222222
93,YOUNG,503,13,2731.538462


In [64]:
county_c_a = county_stats[['County Name', 'County_Count', 'County_Avg_Depth']]
 
county_c_a

,County Name,County_Count,County_Avg_Depth
0,ANDERSON,4,10703.000000
1,ANDREWS,16,11747.500000
2,ARCHER,18,4506.333333
3,ATASCOSA,14,17589.714286
4,BEE,2,23334.000000
...,...,...,...
90,WINKLER,13,9490.846154
91,WISE,8,14551.125000
92,YOAKUM,9,8151.222222
93,YOUNG,13,2731.538462


In [36]:
import geopandas as gpd
import plotly.express as px

In [92]:
county_c_a.loc[county_c_a['County Name'] == 'SAN AUGUSTINE', 'County_Avg_Depth'] = 21420
county_c_a

,County Name,County_Count,County_Avg_Depth
0,ANDERSON,4,10703.000000
1,ANDREWS,16,11747.500000
2,ARCHER,18,4506.333333
3,ATASCOSA,14,17589.714286
4,BEE,2,23334.000000
...,...,...,...
90,WINKLER,13,9490.846154
91,WISE,8,14551.125000
92,YOAKUM,9,8151.222222
93,YOUNG,13,2731.538462


In [95]:
print(county_c_a['County Name'].head())

county_c_a['County Name'] = county_c_a['County Name'].str.upper()

0    ANDERSON
1     ANDREWS
2      ARCHER
3    ATASCOSA
4         BEE
Name: County Name, dtype: object


C:\Users\qkrgu\AppData\Local\Temp\ipykernel_20216\1205628008.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [97]:
texas_counties = gpd.read_file('tx_counties.geojson')  

# 3. 중심점 계산
texas_counties['centroid'] = texas_counties.geometry.centroid
texas_counties['lat'] = texas_counties['centroid'].y
texas_counties['lon'] = texas_counties['centroid'].x

# 4. GeoJSON의 COUNTY 이름 조정
# "Sherman County" -> "SHERMAN"으로 변환 (대문자)
texas_counties['COUNTY'] = texas_counties['COUNTY'].str.replace(' County', '').str.upper()

# 병합
map_data = texas_counties.merge(county_c_a, how='left', left_on='COUNTY', right_on='County Name')

# 5. NaN 값 처리
map_data['County_Count'] = map_data['County_Count'].fillna(0)
map_data['County_Avg_Depth'] = map_data['County_Avg_Depth'].fillna(0)

# 6. 병합 결과 확인
print("병합 결과 상위 5행:")
print(map_data[['COUNTY', 'County Name', 'County_Count', 'County_Avg_Depth', 'lat', 'lon']].head())

병합 결과 상위 5행:
      COUNTY County Name  County_Count  County_Avg_Depth        lat  \
0    SHERMAN     SHERMAN           1.0          3350.000  36.277721   
1   HANSFORD    HANSFORD           6.0         10244.000  36.277409   
2     DALLAM         NaN           0.0             0.000  36.277872   
3  OCHILTREE   OCHILTREE           8.0         12416.125  36.278337   
4   LIPSCOMB    LIPSCOMB           2.0         11530.500  36.277648   

          lon  
0 -101.893445  
1 -101.354548  
2 -102.602198  
3 -100.815632  
4 -100.273127  


C:\Users\qkrgu\AppData\Local\Temp\ipykernel_20216\3521316983.py:4: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.




In [98]:
# 7. 버블 지도 그리기
fig = px.scatter_geo(
    map_data,
    lat='lat',
    lon='lon',
    size='County_Count',  # 우물 개수 -> 원 크기
    color='County_Avg_Depth',  # 평균 깊이 -> 원 색상
    hover_name='County Name',  # 툴팁에 카운티 이름
    hover_data={'County_Count': True, 'County_Avg_Depth': True, 'lat': False, 'lon': False},
    color_continuous_scale='Reds',
    size_max=50,
    scope='usa',
    center={'lat': 31.0, 'lon': -100.0},  # 텍사스 중심
    title='Texas Counties: Well Density (Size) and Average Depth (Color)'
)

fig.update_geos(
    visible=True,
    lataxis_range=[25, 37],
    lonaxis_range=[-107, -93]
)
fig.update_layout(
    margin={"r":0,"t":50,"l":0,"b":0}
)

fig.show()

In [99]:
# county 별 분지
basin_data = pd.read_excel('생산분지.xlsx', sheet_name= 'RegionCounties')

basin_data.head()



,State,County,StateID,CountyID,Region
0,OK,ALFALFA,35,3,Anadarko Region
1,OK,BEAVER,35,7,Anadarko Region
2,OK,BECKHAM,35,9,Anadarko Region
3,OK,BLAINE,35,11,Anadarko Region
4,OK,CADDO,35,15,Anadarko Region


In [100]:
# 카운티랑 생산분지만 뽑음
filtered_basin_data = basin_data[basin_data['State'] == 'TX'][['County', 'Region']]
filtered_basin_data

,County,Region
24,HEMPHILL,Anadarko Region
25,LIPSCOMB,Anadarko Region
26,OCHILTREE,Anadarko Region
27,ROBERTS,Anadarko Region
28,WHEELER,Anadarko Region
...,...,...
310,UPTON,Permian Region
311,VAL VERDE,Permian Region
312,WARD,Permian Region
313,WINKLER,Permian Region


In [101]:
texas_counties = gpd.read_file('tx_counties.geojson')  # 실제 경로로 대체

# 3. 중심점 계산
texas_counties['centroid'] = texas_counties.geometry.centroid
texas_counties['lat'] = texas_counties['centroid'].y
texas_counties['lon'] = texas_counties['centroid'].x

# 4. GeoJSON의 COUNTY 이름 조정 (대문자)
texas_counties['COUNTY'] = texas_counties['COUNTY'].str.replace(' County', '').str.upper()

# 5. county_c_a와 GeoJSON 병합 (geometry 유지)
map_data = texas_counties.merge(county_c_a, how='left', left_on='COUNTY', right_on='County Name')

# 6. filtered_basin_data 병합
# 예: filtered_basin_data = pd.DataFrame({'County': ['HARRIS', 'BEXAR'], 'Region': ['Gulf Coast', 'Permian']})
map_data = map_data.merge(filtered_basin_data, how='left', left_on='County Name', right_on='County')

# 7. NaN 값 처리
map_data['County_Count'] = map_data['County_Count'].fillna(0)
map_data['County_Avg_Depth'] = map_data['County_Avg_Depth'].fillna(0)
map_data['Region'] = map_data['Region'].fillna('Unknown')

# 8. 병합 결과 확인
print("병합 결과 상위 5행:")
print(map_data[['COUNTY', 'County Name', 'County', 'County_Count', 'County_Avg_Depth', 'Region', 'lat', 'lon']].head())

# 9. GeoDataFrame 확인 (필요 시)
print("map_data 타입:", type(map_data))

병합 결과 상위 5행:
      COUNTY County Name     County  County_Count  County_Avg_Depth  \
0    SHERMAN     SHERMAN        NaN           1.0          3350.000   
1   HANSFORD    HANSFORD        NaN           6.0         10244.000   
2     DALLAM         NaN        NaN           0.0             0.000   
3  OCHILTREE   OCHILTREE  OCHILTREE           8.0         12416.125   
4   LIPSCOMB    LIPSCOMB   LIPSCOMB           2.0         11530.500   

            Region        lat         lon  
0          Unknown  36.277721 -101.893445  
1          Unknown  36.277409 -101.354548  
2          Unknown  36.277872 -102.602198  
3  Anadarko Region  36.278337 -100.815632  
4  Anadarko Region  36.277648 -100.273127  
map_data 타입: <class 'geopandas.geodataframe.GeoDataFrame'>


C:\Users\qkrgu\AppData\Local\Temp\ipykernel_20216\2133607878.py:4: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.




In [ ]:
import plotly.graph_objects as go

# 8. 깊이 정규화 (0~1 사이로 색상 강도에 사용)
depth_min = map_data['County_Avg_Depth'].min()
depth_max = map_data['County_Avg_Depth'].max()
if depth_max > depth_min:
    map_data['depth_normalized'] = (map_data['County_Avg_Depth'] - depth_min) / (depth_max - depth_min)
else:
    map_data['depth_normalized'] = 1.0

# 9. 병합 결과 확인
print("병합 결과 상위 5행:")
print(map_data[['COUNTY', 'County Name', 'County_Count', 'County_Avg_Depth', 'Region', 'depth_normalized', 'lat', 'lon']].head())

# 10. 버블 지도 그리기
fig = go.Figure()

# Region별 색상 팔레트 
region_colors = {
    'Eagle Ford Region': ['#1f77b4', '#08306b'],  # 파랑 계열 (밝음 → 어두움)
    'Anadarko Region': ['#ff7f0e', '#7f3b08'],   # 주황 계열
    'Haynesville Region': ['#2ca02c', '#0c4011'], # 초록 계열
    'Permian Region': ['#d62728', '#67000d'],     # 빨강 계열
    'Unknown': ['#7f7f7f', '#3f3f3f']            # 회색 계열
}

# 각 Region별로 데이터 분리 후 플롯
for region, (light_color, dark_color) in region_colors.items():
    region_data = map_data[map_data['Region'] == region]
    if not region_data.empty:
        # 깊이에 따라 색상 보간
        colors = [light_color if d < 0.5 else dark_color for d in region_data['depth_normalized']]
        fig.add_trace(
            go.Scattergeo(
                lat=region_data['lat'],
                lon=region_data['lon'],
                text=region_data['County Name'] + '<br>Count: ' + region_data['County_Count'].astype(str) + '<br>Depth: ' + region_data['County_Avg_Depth'].astype(str),
                mode='markers',
                marker=dict(
                    size=region_data['County_Count'],
                    sizemode='area',
                    sizeref=2. * max(map_data['County_Count']) / (70.**2),  # 원래 크기 유지
                    sizemin=7,
                    color=colors,  # 깊이에 따른 색상 변화
                    line=dict(width=0.5, color='black')
                ),
                name=region
            )
        )

# 지도 레이아웃 설정
fig.update_geos(
    scope='usa',
    lataxis_range=[25, 37],
    lonaxis_range=[-107, -93],
    center={'lat': 31.0, 'lon': -100.0}
)
fig.update_layout(
    title='Texas Counties: Well Density (Size), Production Basin & Depth (Color)',
    margin={"r":0, "t":50, "l":0, "b":0},
    legend_title_text='Production Basin'
)

fig.show()

병합 결과 상위 5행:
      COUNTY County Name  County_Count  County_Avg_Depth           Region  \
0    SHERMAN     SHERMAN           1.0          3350.000          Unknown   
1   HANSFORD    HANSFORD           6.0         10244.000          Unknown   
2     DALLAM         NaN           0.0             0.000          Unknown   
3  OCHILTREE   OCHILTREE           8.0         12416.125  Anadarko Region   
4   LIPSCOMB    LIPSCOMB           2.0         11530.500  Anadarko Region   

   depth_normalized        lat         lon  
0          0.143567  36.277721 -101.893445  
1          0.439016  36.277409 -101.354548  
2          0.000000  36.277872 -102.602198  
3          0.532104  36.278337 -100.815632  
4          0.494150  36.277648 -100.273127  
